In [1]:
import cv2
import numpy as np
import time

In [19]:
def nothing(x):
    pass

# Initialize the webcam feed
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Create a window named trackbars
cv2.namedWindow('Trackbars')

cv2.createTrackbar('L - H', 'Trackbars', 0, 179, nothing)
cv2.createTrackbar('L - S', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('L - V', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('U - H', 'Trackbars', 179, 179, nothing)
cv2.createTrackbar('U - S', 'Trackbars', 255, 255, nothing)
cv2.createTrackbar('U - V', 'Trackbars', 255, 255, nothing)

while True:
    # Start reading the webcam feed frame by frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip the frame horizontally (not required)
    frame = cv2.flip(frame , 1)
    
    # Convert the BGR to HSV image
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get the new values of the trackbar in real time 
    l_h = cv2.getTrackbarPos('L - H', 'Trackbars')
    l_s = cv2.getTrackbarPos('L - S', 'Trackbars')    
    l_v = cv2.getTrackbarPos('L - V', 'Trackbars')
    u_h = cv2.getTrackbarPos('U - H', 'Trackbars')
    u_s = cv2.getTrackbarPos('U - S', 'Trackbars')
    u_v = cv2.getTrackbarPos('U - V', 'Trackbars')
    
    # Set the lower and upper HSV range
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # Filter the image and get the binary mask
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    #stack the mask, original frame and filtered result
    stacked = np.hstack((mask_3, frame, res))
    
    # Show this stacked frame at 40% of the size
    cv2.imshow('Trackbars', cv2.resize(stacked, None, fx=0.4, fy=0.4))
    
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    if key == ord('s'):
        thearray = [[l_h, l_s, l_v], [u_h, u_s, u_v]]
        print(thearray)
        
        np.save('penval', thearray)
        break

cap.release()
cv2.destroyAllWindows()

[[107, 122, 74], [151, 205, 255]]


## Maximizing the Detection Mask and Getting rid of noise

In [2]:
# This variable determines if we want to load color range from memory or use ours
load_from_disk = True

if load_from_disk:
    penval = np.load('penval.npy')

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Creating a 5x5 kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    if load_from_disk:
        lower_range = penval[0]
        upper_range = penval[1]
    else:
        lower_range = np.array([26, 80, 147])
        upper_range = np.array([81, 255, 255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # Perform morphological operations
    mask = cv2.erode(mask, kernel, iterations = 1)
    mask = cv2.dilate(mask, kernel, iterations = 3)
    
    res = cv2.bitwise_and(frame, frame, mask = mask)
    
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack all frames and show it
    stacked = np.hstack((mask_3, frame, res))
    
    cv2.imshow('Trackbars', cv2.resize(stacked, None, fx=0.4, fy=0.4))
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

## Tracking the target pen

In [9]:
load_from_disk = True

if load_from_disk:
    penval = np.load('penval.npy')

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

kernel = np.ones((5, 5), np.uint8)

# This threshold is used to filter noise, the contour area must be 
# bigger than this to qualify as an actual contour.

noiseth = 500

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    if load_from_disk:
        lower_range = penval[0]
        upper_range = penval[1]
    else:
        lower_range = penval[0]
        upper_range = penval[1]
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    mask = cv2.erode(mask, kernel, iterations = 2)
    mask = cv2.dilate(mask, kernel, iterations = 3)
    # Find contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL,
                                          cv2.CHAIN_APPROX_SIMPLE)
    
    # Make sure the contour is present and its bigger than the 
    # threshold value
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
        # Grab the biggest contours in terms of area
        c = max(contours, key = cv2.contourArea)
        
        # Get bounding box coordinates around that contours
        x, y, w, h = cv2.boundingRect(c)
        
        # Draw the bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
    
    cv2.imshow('Image', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cv2.waitKey()

-1

### Drawing with Pen

In [20]:
load_from_disk = True

if load_from_disk:
    penval = np.load('penval.npy')

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 1280)

kernel = np.ones((5, 5), np.uint8)

# Initializing the canvas on which we will draw
canvas = None

# Initialize x1, y1 points
x1, y1 = 0, 0

# Threshold for noise
noiseth = 900

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    if canvas is None:
        canvas = np.zeros_like(frame)
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    if load_from_disk:
        lower_range = penval[0]
        upper_range = penval[1]
    else:
        lower_range = np.array([26, 80, 147])
        upper_range = np.array([81, 255, 255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    mask = cv2.erode(mask, kernel, iterations = 2)
    mask = cv2.dilate(mask, kernel, iterations = 3)

    
    
    # Find contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
        c = max(contours, key = cv2.contourArea)
        x2, y2, w, h = cv2.boundingRect(c)
        cv2.rectangle(frame, (x2, y2), (x2+w, y2+h), (0,255,255),4)
        
        if x1 == 0 and y1 == 0:
            x1, y1 = x2, y2
        else:
            canvas = cv2.line(canvas, (x1, y1), (x2, y2), (255, 0, 0), 4)
        
        x1, y1 = x2, y2
    else:
        # if no contours were detected make x1, y1 0
        x1, y1 = 0, 0
    
    frame = cv2.add(frame, canvas)
    
    # stack both the frames and show it
    stacked = np.hstack((canvas, frame))
    cv2.imshow('Trackbars', cv2.resize(stacked, None, fx=0.6, fy=0.6))
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    
    if k == ord('c'):
        canvas = None


cv2.destroyAllWindows()
cap.release()

### Adding an image wiper

In [5]:
load_from_disk = True
if load_from_disk:
    penval = np.load('penval.npy')

cap = cv2.VideoCapture(0)
# cap.set(3, 1280)
# cap.set(4, 720)

kernel = np.ones((5, 5), np.uint8)

# Making window size adjustable
cv2.namedWindow('image')

# canvas to draw upon
canvas = None

# Initialize x1, y1 points
x1, y1 = 0, 0

# Threshold for noise
noiseth = 800

# Threshold for wiper
wiper_thresh = 4000

# A variable which tells when to clear the canvas
clear = False

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    # Initialize the canvas as black screen
    if canvas is None:
        canvas = np.zeros_like(frame)
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    if load_from_disk:
        lower_range = penval[0]
        upper_range = penval[1]
    else:
        lower_range = np.array([26, 80, 147])
        upper_range = np.array([81, 255, 255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    mask = cv2.erode(mask, kernel, iterations = 2)
    mask = cv2.dilate(mask, kernel, iterations = 3)
    
    # Find contours
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
        c = max(contours, key = cv2.contourArea)
        
        x2, y2, w, h = cv2.boundingRect(c)
        
        # Area of contour
        area = cv2.contourArea(c)
        
        if x1 == 0 and y1 == 0:
            x1, y1 = x2, y2
        else:
            canvas = cv2.line(canvas, (x1, y1), (x2, y2), (255, 0, 0), 4)
        
        x1, y1 = x2, y2
        
        if area > wiper_thresh:
            cv2.putText(canvas, 'Clearing Canvas', (100, 200),
                       cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), cv2.LINE_AA)
            clear = True
    else:
        # If no contours were detected
        x1, y1 = 0, 0
    
    _, mask = cv2.threshold(cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY), 20,
                           255, cv2.THRESH_BINARY)
    foreground = cv2.bitwise_and(canvas, canvas, mask = mask)
    background = cv2.bitwise_and(frame, frame, mask = cv2.bitwise_not(mask))
    frame = cv2.add(foreground, background)
    
    frame = np.hstack((foreground, background, frame))
    cv2.imshow('image', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    
    if clear == True:
        
        time.sleep(1)
        canvas = None
        
        clear = False

cv2.destroyAllWindows()
cap.release()